# FLUENT: A Framework for Efficient Mixed-Protocol Semi-Private Function Evaluation

## Setup

The following cell will set up an environment for experimenting with our FLUENT toolchain. On Colab, the setup takes about 5 minutes.

The code will download binaries for XLS, Yosys, and the Zig compiler toolchain as well as source code from our work. FLUENT is compiled by Zig from source on-the-fly. While Yosys and Zig are fetched from their upstream binary distributions, we modified the XLS codebase and thus provide our own binaries.

In [1]:
# @title
import os
import pathlib

yosys_version = '0.38_93_g84116c9a3'

# download repository
!curl --show-error -L 'https://github.com/encryptogroup/FLUENT/archive/refs/heads/main.tar.gz' | tar xz --strip-components=1

# set up XLS
!curl --show-error -L 'https://github.com/encryptogroup/FLUENT/releases/download/v1.0.0/xls-bin.tar.xz' | tar xJ
xls_bin_path = pathlib.Path('bazel-bin')
xlscc_path = xls_bin_path / 'xls' / 'contrib' / 'xlscc' / 'xlscc'
xls_opt_path = xls_bin_path / 'xls' / 'tools' / 'opt_main'
xls_codegen_path = xls_bin_path / 'xls' / 'tools' / 'codegen_main'
XLSCC = str(xlscc_path.resolve())
XLS_OPT = str(xls_opt_path.resolve())
XLS_CODEGEN = str(xls_codegen_path.resolve())
%env XLSCC={XLSCC}
%env XLS_OPT={XLS_OPT}
%env XLS_CODEGEN={XLS_CODEGEN}

# set up Yosys
!curl -L -O https://repo.anaconda.com/miniconda/Miniconda3-py310_24.1.2-0-Linux-x86_64.sh
!bash Miniconda3-py310_24.1.2-0-Linux-x86_64.sh -b -p conda-env/
conda_prefix_path = pathlib.Path('conda-env')
CONDA_PREFIX = str(conda_prefix_path.resolve())
%env CONDA_PREFIX={CONDA_PREFIX}
!conda-env/bin/conda install -yq -c "litex-hub" yosys={yosys_version}
conda_bin_path = conda_prefix_path / 'bin'
os.environ['PATH'] += os.pathsep + str(conda_bin_path.resolve())

# set up FLUENT
!curl --show-error -L https://ziglang.org/download/0.10.0/zig-linux-x86_64-0.10.0.tar.xz | tar xJ
!cd spfe; ../zig-linux-x86_64-0.10.0/zig build
spfe_bin_path = pathlib.Path('spfe') / 'zig-out' / 'bin'
os.environ['PATH'] += os.pathsep + str(spfe_bin_path.resolve())

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 30.8M    0 30.8M    0     0  5030k      0 --:--:--  0:00:06 --:--:-- 6176k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  227M  100  227M    0     0  4052k      0  0:00:57  0:00:57 --:--:-- 4585k
env: XLSCC=/content/bazel-bin/xls/contrib/xlscc/xlscc
env: XLS_OPT=/content/bazel-bin/xls/tools/opt_main
env: XLS_CODEGEN=/content/bazel-bin/xls/tools/codegen_main
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  128M  100  128M    0     0   185M      0 

## Circuit compilation

The main program responsible for compilation of circuit is `spfe`. We provide the invocation of this executable in the code blocks below. The executable takes the source Verilog/C/C++ file as a command-line argument. Upon successful completion, two files will be generated:

- `<filename>.spfe` contains a textual description of the SPFE circuit.
- `<filename>.spfe.prog` contains programming bits for private modules. This file is generated even when the whole function is public; in this case, it is empty.

You can get an overview of all command-line parameters with `spfe -h`. Here are the most important ones:

- `-t [name]` Top-level function/module (default=main)
- `-l [num]` Use LUTs of size num (default=2) in private modules
- `-L [num]` Use LUTs of size num (default=don't use LUTs) in public modules
- `-c [luc|vuc]` Use a different UC construction (default=luc)

Feel free to change the invocations to try different parameters.

## Introductory examples

Let's take a look at some simple examples from our paper.

### FLUENT in Verilog

The following Verilog code implements a simple 32-bit integer counter and comparator, where the comparator is implemented in a private module.

In [2]:
%%writefile paper_example_verilog.v

(* private *)
module compare ( input  wire [31:0] a, output wire b);
  assign b = a > 4096;
endmodule

(* public *)  // optional
module increment (input  wire [31:0] a, output wire [31:0] c);
  assign c = a + 1;
endmodule

module count_over_threshold (input  wire [31:0] value, input  wire [31:0] counter, output wire [31:0] new_counter);
  wire over_threshold;
  wire [31:0] incremented;
  compare cmp (value, over_threshold);
  increment inc (counter, incremented);
  assign new_counter = over_threshold ? counter : incremented;
endmodule  // main

Writing paper_example_verilog.v


In [3]:
!spfe paper_example_verilog.v -t count_over_threshold
%pfile paper_example_verilog.spfe

compiling public components... compiling private components [1/1] compiling private components [2/1] merging circuits... Object `paper_example_verilog.spfe` not found.


### FLUENT in C

This is the same example implemented in C, made possible with High-Level Synthesis (HLS).

In [ ]:
%%writefile paper_example_c.c

#pragma spfe_private
int compare(int b) {
  return b > 4096;
}

int increment(int a) {
  return a + 1;
}

int count_over_threshold(int value, int counter) {
  return compare(value) ? counter : increment(counter);
}

Writing paper_example_c.c


In [ ]:
!spfe paper_example_c.c -t count_over_threshold
%pfile paper_example_c.spfe

running high-level synthesis with XLS... compiling public components... compiling public components... compiling private components [1/1] compiling private components [2/1] merging circuits... Object `paper_example_c.spfe` not found.


## Car Insurance Example

The car insurance calculator mentioned in our paper is included in our repository. Here, we compile this semi-private function.

**Warning**: This may take mutiple hours as it is a comparatively large circuit.

In [ ]:
!cd spfe/examples/insurance_c/insurance; spfe -I ../libc/musl/include -I ../libc/include/arm-linux-musl -t insurance insurance.c

running high-level synthesis with XLS... running high-level synthesis with XLS... compiling public components... compiling public components... 

## CNN Example

This is a simple Convolutional Neural Network consisting of one convolution layer and one fully connected layer. The convolution layer is private, while the fully connected layer is public.

**Warning**: This may take mutiple hours as it is a comparatively large circuit.

In [ ]:
!cd spfe/examples/cnn/; spfe -t network --dont-optimize-area cnn.cpp